In [1]:
import pandas as pd
import numpy as np
import polars as pl

# Load the dataset
file_path = 'online_retail_II.csv'

# polars
df = pl.read_csv("online_retail_II.csv")

# invoices = pd.read_csv(file_path)

# Display basic info and first rows
invoices.info(), invoices.head()

ComputeError: could not parse `C489449` as dtype `i64` at column 'Invoice' (column number 1)

The current offset in the file is 22122 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `C489449` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [2]:
# Compute revenue using vectorization
invoices["Revenue"] = invoices["Quantity"] * invoices["Price"]

# Clean Description change value to Upper then trim spaces
invoices["Description"] = invoices["Description"].str.upper().str.strip()
# invoices['Description'].str.upper()
# invoices['Description'].str.strip()

# Flag returns / cancellations
invoices["IsReturn"] = invoices["Invoice"].astype(str).str.startswith("C") | (invoices["Quantity"] < 0)

# Show result
print(invoices[["Invoice", "StockCode", "Description", "Quantity", "Price", "Revenue"]].head())

invoices.info(), invoices.head()

  Invoice StockCode                          Description  Quantity Price  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12  6.95   
1  489434    79323P                   PINK CHERRY LIGHTS        12  6.75   
2  489434    79323W                  WHITE CHERRY LIGHTS        12  6.75   
3  489434     22041          RECORD FRAME 7" SINGLE SIZE        48  2.10   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24  1.25   

                                             Revenue  
0   6.956.956.956.956.956.956.956.956.956.956.956.95  
1   6.756.756.756.756.756.756.756.756.756.756.756.75  
2   6.756.756.756.756.756.756.756.756.756.756.756.75  
3  2.102.102.102.102.102.102.102.102.102.102.102....  
4  1.251.251.251.251.251.251.251.251.251.251.251....  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 15 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       ------

(None,
   Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041          RECORD FRAME 7" SINGLE SIZE        48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
        InvoiceDate Price  Customer ID         Country  \
 0  01/12/2009 7:45  6.95      13085.0  United Kingdom   
 1  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 2  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 3  01/12/2009 7:45  2.10      13085.0  United Kingdom   
 4  01/12/2009 7:45  1.25      13085.0  United Kingdom   
 
                    Cleaned Description  Returned/Cancelled Invoices  \
 0  15CM CHRISTMAS GLASS BALL 20 LIGHTS                            0   
 1                   PINK CHERRY LIGHTS                            0   

In [4]:
# Revenue by Country 
#Precompute net revenue
invoices["Net_Revenue"] = invoices["Revenue"].where(~invoices["IsReturn"], 0)

# Precompute net revenue
# invoices["Net_Revenue"] = invoices.apply(
#     lambda row: row["Revenue"] if not row["IsReturn"] else 0, axis=1
# )

# print(invoices)
invoices.info(), invoices.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 16 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Invoice                      1048575 non-null  object 
 1   StockCode                    1048575 non-null  object 
 2   Description                  1044203 non-null  object 
 3   Quantity                     1048575 non-null  int64  
 4   InvoiceDate                  1048575 non-null  object 
 5   Price                        1048575 non-null  object 
 6   Customer ID                  811893 non-null   float64
 7   Country                      1048575 non-null  object 
 8   Cleaned Description          1044203 non-null  object 
 9   Returned/Cancelled Invoices  1048575 non-null  int64  
 10  Missing Description          1048575 non-null  int64  
 11  Returns?                     1048575 non-null  float64
 12  Total Sales                  1048575 non-n

(None,
   Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041          RECORD FRAME 7" SINGLE SIZE        48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
        InvoiceDate Price  Customer ID         Country  \
 0  01/12/2009 7:45  6.95      13085.0  United Kingdom   
 1  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 2  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 3  01/12/2009 7:45  2.10      13085.0  United Kingdom   
 4  01/12/2009 7:45  1.25      13085.0  United Kingdom   
 
                    Cleaned Description  Returned/Cancelled Invoices  \
 0  15CM CHRISTMAS GLASS BALL 20 LIGHTS                            0   
 1                   PINK CHERRY LIGHTS                            0   

In [5]:
# Precompute Returns
invoices["Returns"] = invoices["Revenue"].where(invoices["IsReturn"], 0)

# Precompute returns 
# invoices["Returns"] = invoices.apply(
#     lambda row: row["Revenue"] if row["IsReturn"] else 0, axis=1
# )

invoices.info(), invoices.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 17 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   Invoice                      1048575 non-null  object 
 1   StockCode                    1048575 non-null  object 
 2   Description                  1044203 non-null  object 
 3   Quantity                     1048575 non-null  int64  
 4   InvoiceDate                  1048575 non-null  object 
 5   Price                        1048575 non-null  object 
 6   Customer ID                  811893 non-null   float64
 7   Country                      1048575 non-null  object 
 8   Cleaned Description          1044203 non-null  object 
 9   Returned/Cancelled Invoices  1048575 non-null  int64  
 10  Missing Description          1048575 non-null  int64  
 11  Returns?                     1048575 non-null  float64
 12  Total Sales                  1048575 non-n

(None,
   Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041          RECORD FRAME 7" SINGLE SIZE        48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
        InvoiceDate Price  Customer ID         Country  \
 0  01/12/2009 7:45  6.95      13085.0  United Kingdom   
 1  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 2  01/12/2009 7:45  6.75      13085.0  United Kingdom   
 3  01/12/2009 7:45  2.10      13085.0  United Kingdom   
 4  01/12/2009 7:45  1.25      13085.0  United Kingdom   
 
                    Cleaned Description  Returned/Cancelled Invoices  \
 0  15CM CHRISTMAS GLASS BALL 20 LIGHTS                            0   
 1                   PINK CHERRY LIGHTS                            0   

In [6]:
# changing the Country object as Category
invoices["Country"] = invoices["Country"].astype("category")

In [7]:
# Instead of passing the full DataFrame, subset only the needed columns to make execution faster for 1 M rows dataset
df = invoices[["Country", "Revenue", "Net_Revenue", "Returns"]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   Country      1048575 non-null  category
 1   Revenue      1048575 non-null  object  
 2   Net_Revenue  1048575 non-null  object  
 3   Returns      1048575 non-null  object  
dtypes: category(1), object(3)
memory usage: 25.0+ MB


In [ ]:
# Groupby country (faster with observed=True after Country is change to category) --4454 secs execution with error
country_summary = (
    df.groupby("Country", observed=True)
      .sum()
      .rename(columns={"Revenue": "Gross_Revenue"})
      .reset_index()
)

# Groupby country (faster with observed=True if Country is categorical)
# invoices["Country"] = invoices["Country"].astype("category")
# country_summary = (
#     invoices.groupby("Country", observed=True)[["Revenue", "Net_Revenue", "Returns"]]
#     .sum()
#     .rename(columns={"Revenue": "Gross_Revenue"})
#     .reset_index()
# )

# country_summary = (
#     invoices.groupby("Country", observed=True)[["Revenue", "Net_Revenue", "Returns"]]
#     .sum(numeric_only=True)
#     .rename(columns={"Revenue": "Gross_Revenue"})
#     .reset_index()
# )

#  Faster groupby: one pass
# country_summary = (
#     invoices.groupby("Country")[["Revenue", "Net_Revenue", "Returns"]]
#     .sum()
#     .rename(columns={"Revenue": "Gross_Revenue"})
#     .reset_index()
# )

# this is slow
# Fully vectorized groupby aggregation
# country_summary = invoices.groupby("Country").agg(
#     Gross_Revenue=("Revenue", "sum"),
#     Net_Revenue=("Net_Revenue", "sum"),
#     Returns=("Returns", "sum")
# ).reset_index()

# this code is slow due to loc looping each rows
# country_summary = invoices.groupby("Country").agg(
#     Gross_Revenue=("Revenue", "sum"),
#     Net_Revenue=("Revenue", lambda x: x[~invoices.loc[x.index, "IsReturn"]].sum()),
#     Returns=("Revenue", lambda x: x[invoices.loc[x.index, "IsReturn"]].sum())
# ).reset_index()

print(country_summary)

In [ ]:
# --- Revenue by Product ---
product_summary = invoices.groupby("Description").agg(
    Gross_Revenue=("Revenue", "sum"),
    Net_Revenue=("Revenue", lambda x: x[~invoices.loc[x.index, "IsReturn"]].sum()),
    Returns=("Revenue", lambda x: x[invoices.loc[x.index, "IsReturn"]].sum())
).reset_index()

print(product_summary)

In [ ]:
# --- Sort top results ---
top_countries = country_summary.sort_values("Net_Revenue", ascending=False).head(10)
top_products = product_summary.sort_values("Net_Revenue", ascending=False).head(10)

print(top_countries)
print(top_products)

In [ ]:
# Show results
import plotly.express as px
fig = px.bar(df, x="Country", y="Revenue", title="Revenue by Country")
fig.show() # opens in browser or Jupyter

fig.write_html("revenue_chart.html")
print("Interactive chart saved!")


In [ ]:

# Optional trying to list as string 
combined = (
    invoices["Invoice"].astype(str) + "_" +
    invoices["StockCode"].astype(str) + "_" +
    invoices["Description"].astype(str) + "_" +
    invoices["Quantity"].astype(str) + "_" +
    invoices["InvoiceDate"].astype(str) + "_" +
    invoices["Price"].astype(str) + "_" +
    invoices["Customer ID"].astype(str) + "_" +
    invoices["Country"].astype(str)
)

combined_list = combined.to_list()
# combined it to a combined lsit 
combined_list